### Collecting all the movie links in Cinema Vox Website

In [ ]:
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import datetime

def get_full_date(day_number):
    """Converting day number to format (DD/MM/YYYY)."""
    today = datetime.today()
    day_number = int(day_number)

    # Vérifier si la date est dans le mois suivant
    if day_number < today.day:
        full_date = datetime(today.year, today.month + 1, day_number)
    else:
        full_date = datetime(today.year, today.month, day_number)

    return full_date.strftime("%d/%m/%Y")

def get_film_links(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    film_elements = soup.find_all('a', class_='vignette')
    return [film['href'] for film in film_elements if 'href' in film.attrs]



url= 'https://www.cine-vox.com/films-a-l-affiche/'
films_link = get_film_links(url)
films_link



['https://www.cine-vox.com/film/625829/',
 'https://www.cine-vox.com/film/601002/',
 'https://www.cine-vox.com/film/275719/',
 'https://www.cine-vox.com/film/622088/',
 'https://www.cine-vox.com/film/592500/',
 'https://www.cine-vox.com/film/601348/',
 'https://www.cine-vox.com/film/586620/',
 'https://www.cine-vox.com/film/596644/',
 'https://www.cine-vox.com/film/247659/',
 'https://www.cine-vox.com/film/604926/',
 'https://www.cine-vox.com/film/601986/',
 'https://www.cine-vox.com/film/600912/',
 'https://www.cine-vox.com/film/586040/',
 'https://www.cine-vox.com/film/615074/',
 'https://www.cine-vox.com/film/574120/',
 'https://www.cine-vox.com/film/272899/',
 'https://www.cine-vox.com/film/378846/',
 'https://www.cine-vox.com/film/283427/',
 'https://www.cine-vox.com/film/586364/']

### Creating a new Database containing all the important informations about every movie

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

day_mapping = {
    "Lun.": "lundi",
    "Mar.": "mardi",
    "Mer.": "mercredi",
    "Jeu.": "jeudi",
    "Ven.": "vendredi",
    "Sam.": "samedi",
    "Dim.": "dimanche"
}

def scrape_film_details(driver, films_link):
    all_films = pd.DataFrame()
    
    for i, link in enumerate(films_link):
        driver.get(link)
        
        if i == 0:
            try:
                driver.find_element(By.CLASS_NAME, "didomi-continue-without-agreeing").click()
                driver.find_element(By.ID, "close").click()
                print("Pop-up fermé avec succès.")
            except:
                pass

        # Safely extract film title
        try:
            titre = driver.find_element(By.CLASS_NAME, "ff_titre").text
        except:
            titre = "Not yet revealed"

        # Safely extract film genre
        try:
            genre = driver.find_element(By.CLASS_NAME, "ff_genre").text.lower()
        except:
            genre = "Not yet revealed"

        jour, date = [], []
        for elem in driver.find_elements(By.CLASS_NAME, "hr_jour"):
            text = elem.text.strip()
            if text:
                jour_short = text[:4]  # Extract "lun.", "mar.", etc.
                jour_full = day_mapping.get(jour_short, jour_short)  # Convert into full name
                jour.append(jour_full)

                date_number = re.sub(r"\D", "", text)  # Extract day number
                date_full = get_full_date(date_number)  # Convert to DD/MM/YYYY format
                date.append(date_full)
        
        lst = []
        for j in range(len(date)):
            try:
                driver.find_element(By.XPATH, f'//*[@id="horaires"]/div/div[1]/div/div[{j+1}]').click()
                horaires = [hor.text for hor in driver.find_elements(By.CLASS_NAME, "hor") if hor.text]
            except:
                horaires = []

            lst.append({
                "titre": titre,
                "genre": genre,
                "jour": jour[j],
                "date": date[j],
                "hor": horaires
            })
        
        if lst:
            df = pd.DataFrame(lst).explode("hor")
            all_films = pd.concat([all_films, df])
        
        time.sleep(2)
    
    return all_films

films_data = scrape_film_details(driver, films_link)


Pop-up fermé avec succès.


In [3]:
films_data.head()

,titre,genre,jour,date,hor
0,PINK FLOYD AT POMPEII - MCMLXXII,- genre : concert,lundi,28/04/2025,20h00
0,THE METROPOLITAN OPERA : LES NOCES DE FIGARO,- genre : opera,samedi,26/04/2025,18h55
0,STAR WARS : EPISODE III - LA REVANCHE DES SITH,"- genre : action, aventure, science-fiction",dimanche,27/04/2025,17h30
0,BERNADETTE DE LOURDES : LE SPECTACLE AU CINÉMA,Not yet revealed,dimanche,27/04/2025,16h00
0,DES JOURS MEILLEURS,- genre : comédie dramatique,samedi,26/04/2025,11h00


### Create a function that enables the user to search for movies playing at Cinema Vox by name, genre, or date

In [ ]:
def find_a_movie(all_films):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    print("Welcome to StrasCine \n")
    
    while True:
        choice = input("How would you like to search: \n 1) By genre \n 2) By film name \n 3) By day \n 4) Quit")
        
        if choice == '1':
            print("You chose to view by genre.\n")
            genres = sorted(all_films["genre"].unique().tolist())  # Trier les genres
            for i, g in enumerate(genres):
                print(f"{i+1}) {g}")
            genre_index = int(input("Choose a genre: ")) - 1
            print(all_films[all_films["genre"].str.contains(genres[genre_index], case=False)])
        
        elif choice == '2':
            print("You chose to view by film name.\n")
            titles = sorted(all_films["titre"].unique().tolist())  # Trier les titres
            for i, t in enumerate(titles):
                print(f"{i+1}) {t}")
            title_index = int(input("Choose a film : ")) - 1
            print(all_films[all_films["titre"] == titles[title_index]])
        
        elif choice == '3':
            print("You chose to view by day.\n")
            days = sorted(all_films["jour"].unique().tolist(), key=lambda x: list(day_mapping.values()).index(x))  # Trier par jour
            for i, d in enumerate(days):
                print(f"{i+1}) {d}")
            day_index = int(input("Choose a day : ")) - 1
            print(all_films[all_films["jour"] == days[day_index]])
        
        elif choice == '4':
            print("Thank you for choosing StrasCine. See you soon!")
            break
        else:
            print("Invalid option. Please try again.")

In [9]:
find_a_movie(films_data)

Welcome to StrasCine 

You chose to view by film name.

1) BERNADETTE DE LOURDES : LE SPECTACLE AU CINÉMA
2) BLANCHE NEIGE
3) DES JOURS MEILLEURS
4) DOG MAN
5) DROP GAME
6) L'AMOUR C'EST SURCOTÉ
7) LA FUREUR DE VIVRE
8) LA RÉPARATION
9) MA MÈRE, DIEU ET SYLVIE VARTAN
10) MINECRAFT, LE FILM
11) MOON LE PANDA
12) OZI, LA VOIX DE LA FORÊT
13) PINK FLOYD AT POMPEII - MCMLXXII
14) RAPIDE
15) SINNERS
16) STAR WARS : EPISODE III - LA REVANCHE DES SITH
17) THE AMATEUR
18) THE METROPOLITAN OPERA : LES NOCES DE FIGARO
19) UNTIL DAWN : LA MORT SANS FIN
           titre                                              genre      jour  \
0  BLANCHE NEIGE  - genre : aventure, fantastique, comédie musicale    samedi   
0  BLANCHE NEIGE  - genre : aventure, fantastique, comédie musicale    samedi   
1  BLANCHE NEIGE  - genre : aventure, fantastique, comédie musicale  dimanche   
1  BLANCHE NEIGE  - genre : aventure, fantastique, comédie musicale  dimanche   
2  BLANCHE NEIGE  - genre : aventure, fantastiq